<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex05noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex05noteA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## 準備
----

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりできます．
Python は，機械学習・人工知能やデータサイエンスの分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc  # アニメーションのため
import pandas as pd
import seaborn
seaborn.set()

----
## ロジスティック回帰＋勾配法によるパラメータの最適化 (2)
----




＜ほげお，暗闇を探検す＞

宝の地図を頼りに洞窟探検していたほげお君．
ついに地底の大広間へたどり着きました．地面がすり鉢状になった大きな空間です．
地図によると，この大広間のどこかにある一番低くなった底の地点に，宝箱が埋まっているそうです．

ところが...．ほげお君が「ここまで来たら楽勝ほげ〜」とつぶやいたその瞬間，持っていたライトが消えてしまいました．一面の真っ暗闇．
どうやら電池が切れてしまったようです．「ついてないほげ〜」

しかし，あきらめの悪いほげお君は，大広間の地面に立って考えます．「足元の感覚で，その場所の地面がどっちに傾いてるかは分かるほげな」「なら，下ってる方向にちょっと進んでは立ち止まって足元の傾き調べて，ってのを繰り返せばいけるほげな〜」

ほげお君は無事にお宝をゲットできるでしょうか？

----
### 勾配法とは，最急降下法とは



#### 勾配法

前回説明したように，ロジスティック回帰モデルの学習では，交差エントロピーの値を最小化するようにパラメータを調節します．
平面あてはめ等の最小二乗法のときは，二乗誤差をパラメータで微分して $=0$ とおくことで，最適な（二乗誤差が最小となる）パラメータを解にもつ方程式を導出できました．
しかし，交差エントロピーはより複雑な式をしているため，微分して $=0$ とおいた方程式を解くのは困難です．

このように，目的関数（問題の対象とする関数）を最小に（注）するパラメータを求める問題を解くのが難しい場合，次のような手を使うことがよくあります．

<span style="font-size: 75%">
※注: 目的関数に負号を付けたものをあらためて目的関数とすれば最大化の問題は最小化の問題になるので，ここでは最小化の場合のみを考えます．
<span>

1. パラメータの初期値を適当に定める
1. 現在のパラメータでの目的関数の**勾配**（gradient, パラメータでの微分）の値を求め，勾配を下る方向に（目的関数の値が小さくなる方向に）パラメータを少し修正する
1. 2を繰り返す

このような計算手法を **勾配法** (gradient descent method)といいます．
勾配の値を計算に使いますので，目的関数はパラメータで微分可能でなければいけません．

勾配法の中には様々な手法があるのですが，以下では，その中で最も簡単な手法である，**最急降下法** について説明します．


#### 最急降下法

**最急降下法**(steepest descent method)は，その名の通り「最も急な方向に下っていく」方法です．

ある目的関数が $D$個のパラメータ $w_1, w_2, \ldots, w_D$ を持つとします．パラメータをひとまとめにしたベクトルを $\mathbf{w} = (w_1, w_2, \ldots, w_D)$ と表記することにして，この目的関数を $E(\mathbf{w})$ と書くことにします．
このとき，$E(\mathbf{w})$ に対する最急降下法は次のように表せます．

1. パラメータ $\mathbf{w}$ の初期値を適当に定める
1. 現在のパラメータの値 $\mathbf{w}$ の地点での目的関数の勾配 $\nabla E(\mathbf{w})$ を計算する．
$$
\nabla E(\mathbf{w}) = \left(\frac{\partial E(\mathbf{w})}{\partial w_1}, \frac{\partial E(\mathbf{w})}{\partial w_2}, \ldots, \frac{\partial E(\mathbf{w})}{\partial w_D} \right)
$$
1. 現在の $\mathbf{w}$ と $\nabla E(\mathbf{w})$ の値を用いて，新しいパラメータの値 $\mathbf{w}^{\rm new}$ を次式で計算する．
$$
\mathbf{w}^{\rm new} = \mathbf{w} - \eta \nabla E(\mathbf{w}) \qquad (1)
$$
1. $\mathbf{w} \leftarrow \mathbf{w}^{\rm new}$ とする．
1. 2 から 4 を繰り返す．

式(1)の $\eta$ （ギリシャ文字，「イータ」と発音することが多い）は，あらかじめ定めた小さな正の定数です．機械学習の文脈では，「学習係数」や「学習定数」等と呼ばれます．$\eta$ が大きいと繰り返し1回でのパラメータの変化が大きくなり，$\eta$ が小さいと変化が小さくなります．

このような式と抽象的な説明だけではわけわかめですので，以下のセクションでパラメータ1つや2つの具体例を見てみることにしましょう．

----
### パラメータが一つの場合の具体例


目的関数を
$$
E(w) = w^4 - 11w^3 + 41w^2 - 61w + 40
$$
として，その最小値とそのときの $w$ の値を求める問題を考えましょう．
高校数学の知識があれば手計算で求まりますが，最急降下法でやってみます（注）．

<span style="font-size: 75%">
※注: 手計算では求まらないような複雑な場合だとしても，この問題のような一変数関数の最小化の場合は，もっと他にいい手法があるので，普通は最急降下法は使いません．
<span>

この関数のグラフは下図のようになります．勾配は
$$
\frac{dE(w)}{dw} = 4w^3 - 33w^2+82w-61
$$
です．式(1)より，ある $w$ の地点で勾配 $\frac{dE(w)}{dw}$ が正であれば，$w$の値は負の方向に修正され，勾配が負であれば $w$ の値は正の方向に修正されます．

<img width="75%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/gradescent2.png">

プログラムを実行して最急降下法の計算過程を見てみましょう．

In [ ]:
# 一変数の具体例
def E1(w):
    return w**4 - 11*w**3 + 41*w**2 - 61*w + 40

def dE1dw(w):
    return 4*w**3 - 33*w**2 + 82*w - 61

w = 3.0  # パラメータの初期値
eta = 0.05 # 学習係数

# 最急降下法の繰り返し
for i in range(10):
    print(f'step{i}: w = {w:.4f}, E1(w) = {E1(w):.4f}')
    dw = dE1dw(w)  # 勾配の値の計算
    w -= eta*dw  # パラメータの更新
print(f'step{i}: w = {w:.4f}, E1(w) = {E1(w):.4f}')


繰り返しのたびに $E(w)$ の値が小さくなっていることが分かります．

計算過程をアニメーションでも見てみましょう．

In [ ]:
# 一変数関数を最急降下法で最適化する様子のアニメーションを作成する関数

def genAnim1D(f, dfdx, x0, eta):

    fig, ax = plt.subplots(facecolor="white", figsize=(6, 6))

    # 曲線 f(x) を描く
    xmin, xmax = 0, 6
    xx = np.linspace(xmin, xmax, num=100)
    fxx = f(xx)
    ax.plot(xx, fxx)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(0, 15)

    # アニメーションの各コマを生成
    aList = []
    x, fx = x0, f(x0)
    for i in range(50):
        s = f'step{i}: $({x:.2f}, {fx:.2f})$'
        a1 = ax.plot(x, fx, marker='o', markersize=12, color='red')
        a2 = ax.text(0.4, 0.4, s, size=24)
        a1.append(a2)
        aList.append(a1)
        # 最急降下法
        dx = dfdx(x)
        x -= eta*dx
        fx = f(x)

    anim = animation.ArtistAnimation(fig, aList, interval=300)
    rc('animation', html='jshtml')
    plt.close()

    return anim

In [ ]:
#@title #### 1変数関数の最急降下法による最小化
#@markdown このセルを実行すると，再生ボタンなどが付いたグラフが現れます（**少し時間かかるかも**）．再生してみてね．スライドバーで w0 をいじってからセルを再実行すれば，初期値を変えて実験できます．eta も変えられるよ．

w0 = 3 #@param {type: 'slider', min:0, max:5, step:0.1}
eta = 0.01 #@param [0.01, 0.1] {type: 'raw', allow-input: true}

anim = genAnim1D(E1, dE1dw, w0, eta)
anim

##### ★★ やってみよう ★★

1. `eta = 0.01`で`w0`を以下の3つの値にして実験してみよう．それぞれの場合について，実験の結果がどうなったかを紙媒体にメモしておこう（最小に到達した／最小ではない極小に到達した／その他）．
```
w0 = 5.0
w0 = 0.7
w0 = 2.4
```
1. いろんな `w0`, `eta` の値で実験してみよう．

----
### 最急降下法の性質



上記はパラメータが一つしかない簡単な例ですが，最急降下法の性質についていくつかのことを教えてくれます．

(1) 初期値に依存する／最小解が求まるとは限らない

上記の実験結果を見ると，最急降下法の繰り返しによって $E(w)$ が最小となる $w$ に収束している場合もあれば，それとは異なる $w$ に収束している場合もあります．後者の場合，$E(w)$ が最小になる点ではないものの極小になる点（勾配が$0$となる点）となっています．このように，極小解が複数あるような目的関数に対して最急降下法を適用する場合，パラメータの初期値の選択によっては最小解に収束しない場合があります．




(2) 学習係数を適切に設定する必要がある

最急降下法の式(1)の学習係数 $\eta$ は，繰り返しごとにパラメータの値を修正する大きさを制御します．学習係数があまりに小さいと，解に到達するまでにたくさんの繰り返しが必要となります．逆に大きすぎると，うまく最小化できずに振動してしまったりします．

<span style="font-size: 75%">
※注: 機械学習の問題では目的関数が非常に複雑で極小解が多数あることが一般的ですが，厳密な最小解でなくとも目的関数の値を十分小さくできればok，とする場合が多いです（初期値を変えて何度か試行してみるとか）．
それでも上記のような問題を緩和するために，様々な改良手法が用いられたりします．「よだんだよん」もどうぞ．
<span>

----
### パラメータが二つの場合の具体例


目的関数を
$$
E(w_1, w_2) = 5w_1^2 - 6w_1w_2 + 5w_2^2 - 10w_1 + 6w_2
$$
として，その最小値を与える $(w_1, w_2)$ を最急降下法で求めましょう（注）．

<span style="font-size: 75%">
※注: この関数は凸ですので，本当は最急降下法を使わなくても，勾配 $=0$ とした方程式を解くことで簡単に最小解が求まります．
<span>

勾配を計算すると，次の通りとなります．
$$
\nabla E(w_1, w_2) = \left( \frac{\partial E}{\partial w_1}, \frac{\partial E}{\partial w_2}  \right) = \left( 10w_1-6w_2-10, -6w_1+10w_2+6 \right)
$$

In [ ]:
# 2変数の具体例

def E2(w):
    return 5*w[0]*w[0] - 6*w[0]*w[1] + 5*w[1]*w[1] - 10*w[0] + 6*w[1]

def dE2dw(w):
    return np.array([10*w[0] - 6*w[1] - 10, -6*w[0] + 10*w[1] + 6])

この目的関数をグラフに描くと，次のようになります．
左は， $(w_1, w_2)$ に対する $E(w_1, w_2)$ をそのまま描いた三次元プロット，右は，$E(w_1, w_2)$ の値を等高線としてプロットしたものです．

In [ ]:
# 2変数の具体例のグラフ
fig = plt.figure(facecolor='white', figsize=(12, 6))

# (w1, w2) に対する E(w1, w2) の計算
w1, w2 = np.meshgrid(np.linspace(-10, 10, num=50), np.linspace(-10, 10, num=50))
w1w2 = np.vstack((w1.ravel(), w2.ravel())).T
Ew1w2 = np.array([E2(w) for w in w1w2])
EE = Ew1w2.reshape((w1.shape[0], w2.shape[1]))

# 三次元プロット
elevation = 20
azimuth = -70
ax0 = fig.add_subplot(121, projection='3d')
ax0.plot_wireframe(w1, w2, EE)
ax0.set_xlim(-10, 10)
ax0.set_ylim(-10, 10)
ax0.set_zlim(-10, 1600)
ax0.set_xlabel('$w_1$')
ax0.set_ylabel('$w_2$')
ax0.set_zlabel('$E(w_1, w_2)$')
ax0.view_init(elevation, azimuth)

# 二次元等高線プロット
ax1 = fig.add_subplot(122)
contour = ax1.contour(w1, w2, EE, np.linspace(0, 1600, num=33))
ax1.clabel(contour, fontsize=10)
ax1.set_aspect('equal')
ax1.set_xlabel('$w_1$')
ax1.set_ylabel('$w_2$')
ax1.set_xlim(-10, 10)
ax1.set_ylim(-10, 10)

plt.tight_layout()
plt.show()

1変数のときと同様に最急降下法の計算過程をアニメーションで見てみましょう．

In [ ]:
def genAnim2D(f, dfdx, x0, eta):

    fig, ax = plt.subplots(facecolor="white", figsize=(6, 6))

    # f(x1, x2) の等高線を描く
    xx, yy = np.meshgrid(np.linspace(-10, 10, num=30), np.linspace(-10, 10, num=30))
    XX = np.vstack((xx.ravel(), yy.ravel())).T
    ZZ = np.array([f(x) for x in XX])
    zz = ZZ.reshape((xx.shape[0], xx.shape[1]))
    contour = ax.contour(xx, yy, zz, np.linspace(0, 1600, num=33))
    ax.clabel(contour, fontsize=10)
    ax.set_aspect('equal')
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)

    # アニメーションの各コマを生成
    aList = []
    x = x0
    for i in range(30):
        a1 = ax.plot(x[0], x[1], marker='o', markersize=12, color='red')
        s = f'step{i}: $F = {f(x):.2f}$'
        a2 = ax.text(-9, -8, s, size=24)
        a1.append(a2)
        aList.append(a1)
        # 最急降下法
        dx = dfdx(x)
        x -= eta*dx

    anim = animation.ArtistAnimation(fig, aList, interval=300)
    rc('animation', html='jshtml')
    plt.close()

    return anim

In [ ]:
#@title #### 2変数関数の最急降下法による最小化
#@markdown このセルを実行すると，再生ボタンなどが付いたグラフが現れます（**少し時間かかるかも**）．再生してみてね．スライドバーで w1, w2 をいじってからセルを再実行すれば，初期値を変えて実験できます．eta も変えられるよ．

w1 =  -7#@param {type: 'number'}
w2 = -1 #@param {type: 'number'}
eta = 0.03 #@param [0.03, 0.1] {type: 'raw', allow-input: true}

anim = genAnim2D(E2, dE2dw, [w1, w2], eta)
anim

上記のアニメーションをコマ送り（一時停止してその右隣のボタンを押す）してみましょう．そうすると，パラメータ $\mathbf{w}$ がどのように移動しているのかがよくわかります．二次元のこの例の場合，一度のパラメータ更新で $w_1$ 方向にも $w_2$ 方向にも移動しています．一次元のときは単に 「下っている方へ移動」してるというしかありませんでしたが，この例を見ると，「その地点の勾配ベクトルと逆の向きに進んでいる」ことが分かります．その地点を通る等高線と直交する方向（のうち下る方）ともいえますね．

学習係数 `eta` の値を大きくしてみると，振動する様子も観察できます．

<img width="50%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/gradescent3.png">

----
### よだんだよん


「目的関数の最小解を見つける」という問題は，機械学習の様々な場面に登場します．さらにいうと，機械学習に限らず幅広く様々な科学・工学の分野でも登場します．このような問題は **最適化**（最適化問題，数理最適化） と呼ばれ，一つの研究分野となっています．

最適化の問題とその解法については，後期の科目「最適化の数理I/II」で詳しく学べます．ここで説明しているように目的関数やパラメータが連続な問題だけではなく，離散的・組み合わせ的な場合の最適化問題も扱われます．


ここでは勾配法の中でも最も単純な最急降下法のみを紹介しましたが，機械学習の応用の場面では，勾配法の中でももっと凝った改良手法を用いたり，その他にも様々な連続・離散・組み合わせ最適化の手法を用いたりします．